In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy 
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_admin import Admin
from flask_admin.contrib.sqla import ModelView
from datetime import datetime 
from flask_wtf import FlaskForm
from flask_bcrypt import Bcrypt

app = Flask(__name__)


app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SECRET_KEY'] = 'thisissecret'
app.config['FLASK_ADMIN_SWATCH'] = 'cerulean'
admin = Admin(app, name='Admin', template_mode='bootstrap3')

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "您需要重新登入"
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'
    
class User(UserMixin, db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(50),unique=True)
    email = db.Column(db.String(255),unique=True)
    password = db.Column(db.String(80))
    admin=db.Column(db.Integer)
    
    def __repr__(self):
        return '<User %r>' % self.id
admin.add_view(ModelView(User, db.session))

class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    content = db.Column(db.String(200), nullable=False)
    date_expected = db.Column(db.DateTime, nullable=False)

    def __repr__(self):
        return '<Task %r>' % self.id  
admin.add_view(ModelView(Todo, db.session))


@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route("/")
def home():
    status = "home"
    return render_template('index.html', status=status)

@app.route("/admin")
def admin():
    return render_template("admin/index.html")

@app.route('/login', methods=["GET", "POST"])
def login():
    message=''
    status = "login"
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        print(username,password)

        user = User.query.filter_by(username=username).first()
        if not user:
            print("錯誤的用戶名稱或密碼")
            flash("錯誤的用戶名稱或密碼")
            return redirect(url_for('login'))
        else:
            if password == user.password:
                print("成功登入")
                login_user(user)
                flash("成功登入")
                if user.admin==1:
                    return redirect(url_for('admin'))
                return redirect(url_for('profile'))
            else:
                print("密碼錯誤")
                flash("密碼錯誤")
                return redirect(url_for('login'))           
    else:
        return render_template('login.html',status=status)

@app.route("/profile")
@login_required
def profile():
    status = "profile"
    return render_template('profile.html',username=current_user.username, status=status)

@app.route("/signup", methods=["GET", "POST"])
def signup():
    status = "signup"
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        new_user = User(username=username,email=email,password=password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("成功註冊")
            return redirect(url_for('home'))
        except Exception as e:
            print("註冊失敗",e)
            return 'Fail to add user.'
    else:
        return render_template('signup.html',status=status)

@app.route("/logout")
@login_required
def logout():
    print('用戶登出')
    flash('用戶登出', current_user.username)
    logout_user()
    return redirect(url_for('home')) 


@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

if __name__ == "__main__":
    app.run('localhost', 3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2021 16:16:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:16:43] "GET /login HTTP/1.1" 200 -


In [1]:
import sqlite3 
con = sqlite3.connect('database.db')
cursor = con.cursor()
sql = "CREATE TABLE Todo (id INTEGER, content CHAR(200), date_expected DATE);"
try:
    cursor.execute(sql)
    con.commit()
    print("Table successfully created.")
except Exception as e:
    print("Exception occurred.",e)
cursor.close()
con.close()

Table successfully created.
